In [1]:
import logging
from collections import namedtuple
import random
from numpy.random import choice
import functools

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")

class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [3]:
def _nimsum(state):
    return functools.reduce(lambda a,b : a^b,state)

def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

def optimal(state: Nim) -> Nimply:
    nimsum=_nimsum(state._rows)
    if nimsum==0:
        return (pure_random(state),"pure random")
    else:
        for _ in reversed(range(len(state._rows))):
            if state._rows[_]^nimsum<state._rows[_]:
                return ((_,state._rows[_]-(state._rows[_]^nimsum)),"optimal")
        return (pure_random(state),"pure random")

In [4]:
def number_active_rows(nim):
    return sum([_>0 for _ in nim.rows])


def indexes_active_rows(nim):
    return [_ for _ in range(len(nim.rows)) if nim.rows[_]>0]

def els_active_rows(nim):
    return [_ for _ in nim.rows if _>0]

def els_max_row(nim):
    return max(els_active_rows(nim))

def avg_els_per_row(nim):
    return sum(nim.rows)/len(nim.rows)

def els_min_row(nim):
    return min(els_active_rows(nim))

def els_second_max_row(nim):
    return sorted(els_active_rows(nim),key=lambda a:a,reverse=True)[1]

def index_max_row(nim):
    return sorted(indexes_active_rows(nim),key=lambda a:nim.rows[a],reverse=True)[0]

def index_min_row(nim):
    return sorted(indexes_active_rows(nim),key=lambda a:nim.rows[a])[0]

def index_row_middle_els(nim):
    return sorted(indexes_active_rows(nim),key=lambda a:nim.rows[a])[int(len(indexes_active_rows(nim))/2)]

def index_second_max_row(nim):
    return sorted(indexes_active_rows(nim),key=lambda a:nim.rows[a],reverse=True)[1]

def els_row_middle(nim):
    return nim.rows[sorted(indexes_active_rows(nim),key=lambda a:nim.rows[a])[int(len(indexes_active_rows(nim))/2)]]

def sum_rows_odd(nim):
    return (sum(nim.rows)%2)!=0

def indexes_odd_rows(nim):
    return [_ for _ in range(len(nim.rows)) if not nim.rows[_]%2]

def index_max_odd_row(nim):
    return sorted(indexes_odd_rows(nim),key=lambda a:nim.rows[a],reverse=True)[0]

def index_first_untouched_row(nim,starting_rows):
    for index in range(len(nim.rows)):
        if nim.rows[index]==starting_rows[index]:
            return index

def els_max_odd_row(nim):
    return nim.rows[index_max_odd_row(nim)]

def number_of_rows_with_same_elements_as_starting_elements(nim,starting_rows):
    return sum([nim.rows[_]==starting_rows[_] for _ in range(len(nim.rows))])

def only_odd_rows(nim):
    return all([not nim.rows[_]%2 for _ in indexes_odd_rows(nim)])

def hard_coded_strategy(nim,max_start,starting_rows,rows):
    if number_active_rows(nim)<2:
        return ((index_max_row(nim),els_max_row(nim)),"first rule")
    if number_of_rows_with_same_elements_as_starting_elements(nim,starting_rows)>=rows/2:
        return ((index_first_untouched_row(nim,starting_rows), 1),"second rule")
    if number_active_rows(nim)==2:
        return ((index_max_row(nim),els_second_max_row(nim)),"third rule")
    if avg_els_per_row(nim)>(max_start/4) and avg_els_per_row(nim)<(max_start/2):
        return ((index_max_row(nim),els_max_row(nim)//2),"fourth rule")
    if number_active_rows(nim)>rows/2:
        return ((index_row_middle_els(nim),els_row_middle(nim)),"fifth rule")
    if sum_rows_odd(nim):
        return ((index_max_odd_row(nim),els_max_odd_row(nim)),"sixth rule")
    if only_odd_rows(nim) and number_active_rows(nim)>1:
        return ((index_max_row(nim),els_second_max_row(nim)),"seventh rule")

    

    else:
        return (pure_random(nim),"pure random")

In [5]:
wins=0
logging.getLogger().setLevel(logging.DEBUG)
starting=random.choice([True,False])
player=0
NUM_GAMES=20
NUM_ROWS=11
MAX_START=21
for _ in range(NUM_GAMES):
    nim=Nim(NUM_ROWS)
    starting_rows=[_ for _ in nim.rows]
    logging.debug(f"In this game I'm player #{0 if starting else 1}")
    logging.debug(f"Game starting, AM I STARTING??? {starting} with these rows {nim} and this nimsum {_nimsum(nim.rows)}")
    while nim:
        if starting!=player:
            ply=hard_coded_strategy(nim,MAX_START,starting_rows,NUM_ROWS)
        else:
            ply=(gabriele(nim),"pure randommmone")
        nim.nimming(ply[0])
        logging.debug(f"After player {player} play {ply} move now rows are {nim} and nimsum is {_nimsum(nim.rows)}")
        player=1-player
    winner=1-player
    won=(winner==0 and starting) or (winner==1 and not starting)
    #myPlayer.evaluate_game(won)
    if won:
        wins+=1
    starting=random.choice([True,False])

logging.debug(f"After {NUM_GAMES} my player won {wins} games!")

DEBUG:root:In this game I'm player #0
DEBUG:root:Game starting, AM I STARTING??? True with these rows <1 3 5 7 9 11 13 15 17 19 21> and this nimsum 23
DEBUG:root:After player 0 play ((0, 1), 'second rule') move now rows are <0 3 5 7 9 11 13 15 17 19 21> and nimsum is 22
DEBUG:root:After player 1 play (Nimply(row=1, num_objects=3), 'pure randommmone') move now rows are <0 0 5 7 9 11 13 15 17 19 21> and nimsum is 21
DEBUG:root:After player 0 play ((2, 1), 'second rule') move now rows are <0 0 4 7 9 11 13 15 17 19 21> and nimsum is 20
DEBUG:root:After player 1 play (Nimply(row=2, num_objects=4), 'pure randommmone') move now rows are <0 0 0 7 9 11 13 15 17 19 21> and nimsum is 16
DEBUG:root:After player 0 play ((3, 1), 'second rule') move now rows are <0 0 0 6 9 11 13 15 17 19 21> and nimsum is 17
DEBUG:root:After player 1 play (Nimply(row=3, num_objects=6), 'pure randommmone') move now rows are <0 0 0 0 9 11 13 15 17 19 21> and nimsum is 23
DEBUG:root:After player 0 play ((4, 1), 'second 